
# EEG Data Analysis Assignment


In this assignment you will analyze the EEG data that you collected yourself. There are different approaches to preprocessing. Here we will model our pipeline after this recent paper (check Methods): https://pmc.ncbi.nlm.nih.gov/articles/PMC10659264/pdf/nihpp-2023.11.07.566051v2.pdf.

## Part 1: Put all data in BIDS format

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from mne_bids import BIDSPath, write_raw_bids, read_raw_bids

# Define directories:
project_dir = os.getcwd()
print(project_dir)
raw_dir = os.path.join(project_dir, 'raw')
print(raw_dir)
bids_dir = os.path.join(project_dir, 'bids')

# Define event IDs:
event_id = {
    'A/standard': 1,
    'A/oddball':  2,
    'B/standard': 3,
    'B/oddball':  4,
}

# subject ID
subjects = ["01", '02', '03', '04', '05','06']

🔥 Please loop over filenames and put all raw data in BIDS format (look up what this means). You can use the functions ```BIDSPath``` and ```write_raw_bids```. We need to take care of a couple of things.

First, after loading the raw EEG data, we need to change channels 'M1', 'M2', 'EXG7' and 'EXG8' to type 'misc', and channels 'UP', 'DOWN', 'LEFT' and 'RIGH' to type 'eog'. We also need to set the right montage ('biosemi64').

Second, observe that ```write_raw_bids``` takes the "events" and "event_id" parameters. During EEG acquisition we always sent the same trigger (2) at every sound presentation. However, we would like to include information about state and stimulus, as per the dictionary called "event_id" (see previous cell). Please make sure to: (i) find all events in the eeg file, (ii) read information about state and stimulus ID in the corresponding "*_events.tsv" file, (iii) update the events according to "event_id", (iv) pass the "events" and "event_id" in ```write_raw_bids```.

Third, look at all the json sidecar files, and check whether all information is correct. 


In [ ]:
def rawtobids(raw_dir, bids_dir, subjects, event_id):
    """
    Converteer alle .bdf bestanden in raw_dir naar BIDS in bids_dir.
    Voor elk bestand:
      - zet opgegeven kanaaltypes (misc, eog)
      - zet montage biosemi64
      - vind STIM-events in raw
      - lees corresponderende *_events.tsv (zelfde basename + '_events.tsv')
      - filter phase == "2" en map (state, frequency) naar de juiste event_id
      - zet de derde kolom van de events-array naar die nieuwe codes (alleen voor de fase2-triggers)
      - schrijf naar BIDS met write_raw_bids(..., events_data=events_array, event_id=event_id)
    """
    event_id = event_id
    bdf_files = sorted(glob.glob(os.path.join(raw_dir, "*.bdf")))
    tsv_files = sorted(glob.glob(os.path.join(raw_dir, "*.tsv")))
    print("Gevonden .bdf bestanden:", bdf_files)
    print("Gevonden .bdf bestanden:", len(bdf_files))
    print("Gevonden .tsv bestanden:", tsv_files)
    print("Gevonden .tsv bestanden:", len(tsv_files))

    for i, bdf_file in enumerate(bdf_files):
        print("\nVerwerken:", bdf_file)
        # lees raw bdf files in
        raw = mne.io.read_raw_bdf(bdf_file, preload=True)

        # Types omzetten op basis van kanaal
        misc = ['M1', 'M2', 'EXG7', 'EXG8']
        raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
        eog = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        raw.set_channel_types({ch: 'eog' for ch in eog if ch in raw.ch_names})

        # montage biosemi64
        raw.set_montage('biosemi64', on_missing='ignore')

        # -- vind events in raw (STIM/trigger-kanaal). Gebruik veilige find_events call --
        try:
            all_events = mne.find_events(raw, shortest_event=1, verbose=False)
        except Exception as exc:
            print(f"  Kan events niet vinden met mne.find_events() voor {bdf_file}: {exc}")
            all_events = np.empty((0, 3), dtype=int)

        # Filter originele triggers: we verwachten tijdens stimuli trigger value == 2
        stim_mask = (all_events[:, 2] == 2) if all_events.size else np.array([], dtype=bool)
        stim_events = all_events[stim_mask] if stim_mask.size else np.empty((0, 3), dtype=int)
        print(f"  Aantal gevonden stimulus-triggers (waarde==2): {len(stim_events)}")

        # Extract prefix from BDF file (e.g., '1_2_' from '1_2_eeg.bdf')
        prefix = os.path.basename(bdf_file).split('_eeg.bdf')[0] + '_'
        events_tsv_path = None

        # Search for matching events file
        for file_path in tsv_files:
            file_name = os.path.basename(file_path)
            if file_name.startswith(prefix) and file_name.endswith('_events.tsv'):
                events_tsv_path = file_path
                break

        if events_tsv_path is None:
            print(f"  Waarschuwing: events file niet gevonden voor prefix {prefix}. Schrijf BIDS zonder aangepaste events.")
            events_array_to_write = None
        else:
            # Read and filter the events file
            df = pd.read_csv(events_tsv_path, sep='\t', dtype=str)
            filtered = df[df["phase"].str.strip() == "2"].copy()
            print(f"  Aantal events in {os.path.basename(events_tsv_path)} met phase==2: {len(filtered)}")

            # Maak een nieuwe lijst voor de aangepaste event codes zoals in de dictionary
            new_event_codes = []
            if not filtered.empty:
                # zet veilig om naar floats/integers waar nodig
                # (we volgen hetzelfde logic als in jouw oorspronkelijke code)
                for st_str, fq_str in zip(filtered["state"], filtered["freq"]):
                    try:
                        st = float(str(st_str).strip())
                    except:
                        st = np.nan
                    try:
                        fq = float(str(fq_str).strip())
                    except:
                        fq = np.nan

                    if st == 1:  # reeks A
                        if fq == 2000:
                            new_event_codes.append(event_id["A/standard"])
                        else:
                            new_event_codes.append(event_id["A/oddball"])
                    elif st == -1:  # reeks B
                        if fq == 1000:
                            new_event_codes.append(event_id["B/standard"])
                        else:
                            new_event_codes.append(event_id["B/oddball"])
                    else:
                        new_event_codes.append(0)  # ongedefinieerd -> code 0 (volgt jouw eerdere aanpak)

            n_triggers = len(stim_events)
            n_filtered = len(new_event_codes)

            # code om errors te voorkomen: (dit was een issue een paar keer)
            # 1. Het controleert of er stim-events zijn gevonden in de raw data.
            # 2. Als er geen stim-events zijn, wordt events_array_to_write op None gezet.
            # 3. Als er wel stim-events zijn, maar het aantal gevonden stim-events verschilt van het aantal gefilterde phase2-rows,
            #    wordt er een waarschuwing gegeven en worden de arrays afgestemd op de minimale lengte.
            if n_triggers == 0:
                print("  Geen stim-events gevonden in raw; geen events meegeven aan write_raw_bids.")
                events_array_to_write = None
            else:
                if n_triggers != n_filtered:
                    print(f"  Let op: aantal gevonden stim-events ({n_triggers}) ≠ aantal gefilterde phase2-rows ({n_filtered}).")
                    print("  Ik zal de arrays afstemmen op de minimale lengte (geen fouten raise).")
                n_use = min(n_triggers, n_filtered)
                if n_use == 0:
                    events_array_to_write = None
                else:
                    events_array_to_write = stim_events[:n_use].copy()
                    events_array_to_write[:, 2] = np.array(new_event_codes[:n_use], dtype=int)
                    print(f"  Events array klaargezet met {n_use} events (kolom 3 aangepast).")

        # BIDSPath maken (subject index i -> subjects[i])
        bids_path = BIDSPath(subject=subjects[i],
                                root=bids_dir,
                                datatype='eeg',
                                extension='.bdf',
                                suffix='eeg',
                                task='experiment')

        # Schrijf de events naar BIDS
        if events_array_to_write is None:   # als events ontbreken alleen bdf schrijven
            write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')
            print(f"  {os.path.basename(bdf_file)} geschreven naar BIDS (zonder events).")
        else:
            write_raw_bids(raw,
                            bids_path=bids_path,
                            events = events_array_to_write,
                            event_id=event_id,
                            overwrite=True,
                            allow_preload=True,
                            format='EDF')
            print(f"  {os.path.basename(bdf_file)} geschreven naar BIDS (met events).")
    print("\nKlaar met alle bestanden inbids zetten.")

# run functie
rawtobids(raw_dir=raw_dir, bids_dir=bids_dir, subjects=subjects, event_id=event_id)

👉 **Question:** Why would we bother with storing our data in BIDS format?

**Answer:**


## Part 2: Preprocess example data set

🔥 Please take an example block and load the eeg and events data (from the BIDS directory).

In [ ]:
#load the eeg and events data (from the BIDS directory).
# take an example block and load the eeg and events data from the BIDS directory

# path to first participant file
bids_path = BIDSPath(subject=subjects[0],
                             root=bids_dir,
                             datatype='eeg',
                             extension='.edf',
                             suffix='eeg',
                             task='experiment')
# load the data from the first participant
raw = read_raw_bids(bids_path=bids_path, verbose=False) 
print(bids_path)

if raw is None:
    print("Failed to load raw data from BIDS path:", bids_path)
elif raw:
    print("Raw data successfully loaded from BIDS path of participant:",str(str(bids_path).split('_task-experiment_eeg.bdf'))[70:76]  )

# isolate events from the raw data
events, event_id = mne.events_from_annotations(raw)
if not events.size:
    print("Geen events gevonden in de data.")
elif events.size:
    print("Events geladen als events:.")
print('events',events)
print('event_id',event_id)

🔥 Please plot the raw data. Show examples of (i) clean data, (ii) blinks (look at eog channels), and (iii) muscle artefacts. Print the shape of the data, and indicate what the dimensions correspond to. Print all channel names. Print all channel types.

In [17]:
# Your code goes here. Please add comments.
# Irem

# Plot the raw data
raw.plot()

#Printing the shape of the data
print(f"Shape of the data: {raw._data.shape}")
print("The dimensions correspond to (n_channels, n_times)")

#Printing the channel names
print(f"Channel names: {raw.ch_names}")

#Printing the channel types
print(f"Channel types: {raw.get_channel_types()}")


#Example of clean data
raw.plot(start=0, duration=10, title='Clean Data')

#Example of the blinks looking at the eog channels
raw.plot(start=20, duration=10, title='Blinks (EOG Channels)')

#Example of muscle artefacts
raw.plot(start=40, duration=10, title='Muscle Artefacts')

raw.plot()

NameError: name 'raw' is not defined

👉 **Question:** What kind of information can you extract just from visually inspecting raw EEG traces?

**Answer:**

🔥 Please set the channels 'M1', 'M2', 'EXG7' and 'EXG8' to type 'misc', and channels 'UP', 'DOWN', 'LEFT' and 'RIGH' to type 'eog'. Also set the right montage, and plot this. 

PS: Yes, it is annoying that we did this before. This information did end up correctly in the json sidecar files (check this), but not in the *eeg.bdf files.

In [18]:
# Your code goes here. Please add comments.
# Irem
#The bids_path for raw data (dont think this is correct)
raw = mne.io.read_raw_edf(bids_path, preload=True)

# Types omzetten op basis van kanaal
misc = ['M1', 'M2', 'EXG7', 'EXG8']
raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
eog = ['UP', 'DOWN', 'LEFT', 'RIGH']
raw.set_channel_types({ch: 'eog' for ch in eog if ch in raw.ch_names})

# montage biosemi64
raw.set_montage('biosemi64')

#plot of the raw data as whole
raw.plot()

Extracting EDF parameters from C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\Code-Data-Science\Code-Data-Science\bids\sub-01\eeg\sub-01_task-experiment_eeg.edf...
EDF file detected


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Gebruiker\\Desktop\\Code Data Science\\Code-Data-Science\\Code-Data-Science\\Code-Data-Science\\bids\\sub-01\\eeg\\sub-01_task-experiment_eeg.edf'

🔥 Please re-reference the EEG data to the average of the two mastoids.

In [19]:
# Your code goes here. Please add comments.
# Roderik
##checkt of er er M1 en M2 in raw zitten
if 'M1' in raw.ch_names and 'M2' in raw.ch_names:
    raw.set_channel_types({'M1': 'eeg', 'M2': 'eeg'})  # maakt het type van M1 en M2 eeg

    # zet het gemiddelde als reference
    raw.set_eeg_reference(ref_channels=['M1', 'M2'])
    print("Re-referenced to average of M1 and M2.")
else:
    print("M1/M2 not found in data. Cannot apply mastoid reference.")

NameError: name 'raw' is not defined

👉 **Question:** Why would you want to re-reference the EEG data? What are common approaches? What are advantages and disadvantages of each?

**Answer:**

🔥 Please combine eog channels into bipolar horizontal ("HEOG") and vertical ("VEOG") EOG derivations. Check if the resulting channel type is correct.

In [ ]:
# Your code goes here. Please add comments.
# Roderik
#combineert de verschillende oog componenten
raw = mne.set_bipolar_reference(raw,
                                anode='LEFT', cathode='RIGHT',
                                ch_name='HEOG', copy=False)

raw = mne.set_bipolar_reference(raw,
                                anode='UP', cathode='DOWN',
                                ch_name='VEOG', copy=False)

# na het combineren worden  het resultaat van channel type verandert naar eog
raw.set_channel_types({'HEOG': 'eog', 'VEOG': 'eog'})

# print ['HEOG', 'VEOG'] als deze erin zitten samen met type van channel
print("New EOG channels added:")
for ch, ch_type in zip(raw.ch_names, raw.get_channel_types()):
    if ch in ['HEOG', 'VEOG']:
        print(f"  {ch}: {ch_type}")
        

👉 **Question:** Why do we take the difference between for example the UP and DOWN electrode?

**Answer:** 

🔥 Please apply a bandpass filter. Look up what a bandpass filter is, and use common cutoffs for EEG.

In [ ]:
# Your code goes here. Please add comments.
# Roderik
import matplotlib.pyplot as plt
# Your code goes here. Please add comments.
# MNE heeft deze functie om bandpass filters toe te passen
raw.filter(l_freq=0.1, h_freq=30.0)

#Check resultaat
# maakt kopie van raw
raw_copy = raw.copy()

# power spectrum voor het filter
fig_before = raw_copy.plot_psd(fmax=60, show=False)
plt.suptitle("Power Spectral Density (before filtering)", fontsize=14)

# filter
raw_copy.filter(l_freq=0.1, h_freq=30.0)

# power spectrum na het filter
fig_after = raw_copy.plot_psd(fmax=60, show=False)
plt.suptitle("Power Spectral Density (after 0.1–30 Hz bandpass)", fontsize=14)

plt.show()

👉 **Question:** Why should we filter EEG data? What type of temporal filters are typically applied, and what are common cut-offs? 

**Answer:**

🔥 Please downsample the EEG data to 250Hz.

In [ ]:
# Your code goes here. Please add comments.
# Mernan
# Downsample de data naar 250 Hz

raw_downsampled = raw.resample(250, npad='auto')

# Om te controleren of het downsamplen gelukt is 
print("Original sampling rate:", raw.info['sfreq'])
print("Resampled sampling rate:", raw_downsampled.info['sfreq'])

👉 **Question:** What are the advantages of downsampling, and why does 250Hz makes sense?

**Answer:**

🔥 Please perform independent component analysis and plot the components.

In [13]:
# Your code goes here. Please add comments.
# Sam
from mne.preprocessing import ICA

# Set up
n_components = 15 
ica = ICA(n_components=n_components, random_state=97, max_iter=800)
ica.fit(raw_downsampled)
ica.fit

# Plot the components
print("Plotting ICA components...")
ica.plot_components(show=True)

# plot the sources (time series)
ica.plot_sources(raw_downsampled, show=True)

# Plot the properties of each component to help identify artifacts
ica.plot_properties(raw_downsampled, picks=range(n_components))

NameError: name 'raw_downsampled' is not defined

👉 **Question:** What does this plot show?  

**Answer:** 


🔥 Please remove components associated with blinks and eye movements. Include diagnostic figures (check with plotting methods exists for ica objects).

In [ ]:
# Your code goes here. Please add comments.
# Mernan
#Remove blinks and eye movement artifacts with ICA
from mne.preprocessing import ICA

# Fit ICA
ica = ICA(n_components=15, random_state=97, max_iter='auto') #15 componenten gekozen omdat dat de meest gebruikte is kan veranderd worden maar het is tussen 10-20 is het volgens het internet
ica.fit(raw_downsampled) # fit op resampled data

ica.plot_components() # bekijk de componenten

# Zoek EOG-artifacten 
## eog_indices kijkt naar de componenten die corresponderen met oogbewegingen en die we willen verwijderen
## eog_scores geeft een score aan hoe goed elke component correleert met de EOG dus hoe hoger de score hoe beter die component correleert met oogbewegingen
eog_indices, eog_scores = ica.find_bads_eog(raw_downsampled, ch_name=['VEOG', 'HEOG']) 

# Exclude EOG artifacts
ica.exclude = eog_indices

#Plot de eigenschappen van de uitgesloten componenten
ica.plot_scores(eog_scores)
ica.plot_properties(raw_downsampled, picks=ica.exclude)
plt.show()

# pas ICA toe op de resampled data
raw_cleaned = raw_downsampled.copy()
ica.apply(raw_cleaned)

👉 **Question:** What do these plot show?  

**Answer:** 

🔥 Please plot the same bit of data, before and after ica-based artefact removal.

In [ ]:
# Your code goes here. Please add comments.
# Irem


👉 **Question:** What kinds of artifacts can ICA help to remove in EEG data? Why do we want to do this? 

**Answer:**

🔥 Please epoch the cleaned data (make sure to use "event_id"), by cutting out segments from 500ms before each sound to 1000ms after. Plot the resulting epochs object.

In [ ]:
# Your code goes here. Please add comments.
# Mernan
# Epoching de data rond 500ms voor en 1000ms na stimulus
# We gebruiken de events die we in de BIDS stap hebben toegevoegd
events, event_id = mne.events_from_annotations(raw_cleaned)

# Maak epochs rond de events
epochs = mne.Epochs(raw_cleaned,
                    events=events,
                    event_id=event_id,
                    tmin=-0.5, tmax=1.0,  # 500 ms voor tot 1000 ms na stimulus
                    baseline=(None),   # geen baseline correctie kunnen we nog eventueel veranderen en wel een baseline doen
                    preload=True)   

# Om te kijken hoeveel epochs er zijn
print("Number of epochs:", len(epochs))

# Bekijk de eerste 20 epochs
epochs.plot(n_epochs=20)


👉 **Question:** Why do we extract epochs relative to events?  

**Answer:**


## Part 3: Preprocess all the data

🔥 Please put all the code we wrote so far in function called ```preprocess_eeg_data```. The function should take "bids_path" as input and return "epochs", You can then loop across all data and concatenate all the epochs.

In [ ]:
# Your code goes here. Please add comments.

## Part 4: Analyze the data

🔥 Please define the following sensor groupings: frontal, central, temporal, parietal and occipital.

In [ ]:
# define sensor groupings:
frontal = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7',
           'Fp2', 'AF8', 'AF4', 'F2', 'F4', 'F6', 'F8']
central = ['FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'CP5', 'CP3', 'CP1',
           'FC6', 'FC4', 'FC2', 'C2', 'C4', 'C6', 'CP6', 'CP4', 'CP2']
temporal = ['FT7', 'T7', 'TP7', 'FT8', 'T8', 'TP8',]
parietal = ['P1', 'P3', 'P5', 'P7', 'P9', 
            'P2', 'P4', 'P6', 'P8', 'P10']
occipital = ['PO7', 'PO3', 'O1',
             'PO8', 'PO4', 'O2']